In [ ]:
import sys
import os
import cv2
import numpy as np
import pandas as pd
import json
from datetime import datetime
from tensorflow.keras.models import load_model
from PyQt5.QtWidgets import (
    QMainWindow, QApplication, QWidget, QVBoxLayout, QLabel, QProgressBar,
    QTextEdit, QMenuBar, QMenu, QAction, QFileDialog, QMessageBox, QDialog,
    QPushButton, QTableWidget, QTableWidgetItem, QHeaderView, QHBoxLayout
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, pyqtSlot
from PyQt5.QtGui import QPixmap, QImage, QIcon, QFont
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.cell import MergedCell  
from PyQt5.QtWidgets import QTableWidget, QTableWidgetItem, QHeaderView, QHBoxLayout

#Базовый класс для покадрового и файлового потоков
class BaseVideoProcessor:
    update_frame = None  # Будет установлено из потока
    update_result = None  # Будет установлено из потока
    MIN_CONFIDENCE_THRESHOLD = 50.0  # Минимальный порог уверенности в %
    def __init__(self, model, image_size=(64, 64), sequence_length=16):
        self.model = model
        self.image_size = image_size
        self.sequence_length = sequence_length
        self.processing = True
    
    def analyze_video(self, video_path, emit_frame=False, emit_result=False):
        """Общий метод для анализа видео"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return {
                "filename": os.path.basename(video_path),
                "status": "error",
                "total_frames": 0,
                "violence_frames": 0,
                "max_confidence": 0.0
            }
            # Проверка на статичное видео
        ret, frame1 = cap.read()
        ret, frame2 = cap.read()
        if not ret or np.array_equal(frame1, frame2):
            cap.release()
            return {
                "filename": os.path.basename(video_path),
                "status": "static",
                "total_frames": 0,
                "violence_frames": 0,
                "max_confidence": 0.0
            }
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:  # Защита от деления на ноль
            fps = 30  # Значение по умолчанию
        frame_sequence = []
        violence_timestamps = []
        max_confidence = 0.0
        
        video_result = {
            "filename": os.path.basename(video_path),
            "total_frames": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
            "violence_frames": 0,
            "violence_percentage": 0.0,
            "timestamps": [],
            "max_confidence": 0.0
        }
        
        while cap.isOpened() and self.processing:
            ret, frame = cap.read()
            if not ret:
                break
            
            if emit_frame:
                self.update_frame.emit(frame.copy())
            
            # Обработка кадров
            resized = cv2.resize(frame, self.image_size)
            normalized = resized / 255.0
            frame_sequence.append(normalized)
            if len(frame_sequence) == self.sequence_length:
                input_data = np.expand_dims(frame_sequence, axis=0)
                prediction = self.model.predict(input_data, verbose=0)[0]
                is_violence = np.argmax(prediction) == 1
                confidence = float(prediction[1]) * 100
                
                # Обновляем максимальную точность
                if confidence > max_confidence:
                    max_confidence = confidence
                    video_result["max_confidence"] = max_confidence
                
                # Обрабатываем только если уверенность выше порога
                if is_violence and confidence >= self.MIN_CONFIDENCE_THRESHOLD:
                    current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                    timestamp = (current_frame - self.sequence_length) / fps
                    violence_timestamps.append(timestamp)
                    video_result["violence_frames"] += self.sequence_length
                    
                    if emit_result:
                        self.update_result.emit(
                            f"Насилие на {timestamp:.1f} сек. (Точность: {confidence:.2f}%)",
                            violence_timestamps.copy(),
                            video_result
                        )
                
                frame_sequence = []
        
        cap.release()
        
        # Рассчитываем процент кадров с насилием
        if video_result["total_frames"] > 0:
            video_result["violence_percentage"] = (
                (video_result["violence_frames"] / video_result["total_frames"]) * 100
            )
        else:
            video_result["violence_percentage"] = 0.0
            
        video_result["max_confidence"] = max_confidence
        video_result["timestamps"] = violence_timestamps
        
        return video_result
    
# Класс для покадрового анализа (FrameAnalysisThread)
class FrameAnalysisThread(QThread):
    update_progress = pyqtSignal(int, int, str)
    update_result = pyqtSignal(str, list, dict)
    update_frame = pyqtSignal(np.ndarray)
    video_file_processed = pyqtSignal()
    
    def __init__(self, video_path, model, image_size, sequence_length):
        super().__init__()
        self.video_path = video_path
        self.processor = BaseVideoProcessor(model, image_size, sequence_length)
        
        # Связываем сигналы процессора с сигналами потока
        self.processor.update_frame = self.update_frame
        self.processor.update_result = self.update_result
    
    def run(self):
        video_files = [f for f in os.listdir(self.video_path)
                     if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        total_videos = len(video_files)
        
        for i, video_file in enumerate(video_files):
            if not self.processor.processing:
                break
                
            self.update_progress.emit(i+1, total_videos, f"Обработка {video_file}")
            full_path = os.path.join(self.video_path, video_file)
            
            # Используем метод из BaseVideoProcessor
            self.processor.analyze_video(full_path, emit_frame=True, emit_result=True)
            self.video_file_processed.emit()

# Класс для общего анализа (класс-поток для использования базового класса)
class FileAnalysisThread(QThread):
    #cap = cv2.VideoCapture(video_path)
    update_progress = pyqtSignal(int, int, str)
    video_processed = pyqtSignal(dict)
    update_frame = pyqtSignal(np.ndarray)

    def __init__(self, video_path, model, image_size, sequence_length):
        super().__init__()
        self.video_path = video_path
        self.processor = BaseVideoProcessor(model, image_size, sequence_length)
        self.processor.update_frame = self.update_frame
    
    def run(self):
        video_files = [f for f in os.listdir(self.video_path)
                     if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        total_videos = len(video_files)
        
        for i, video_file in enumerate(video_files):
            if not self.processor.processing:
                break
                
            self.update_progress.emit(i+1, total_videos, f"Обработка {video_file}")
            full_path = os.path.join(self.video_path, video_file)
            # Передаем emit_frame=True для показа кадров
            result = self.processor.analyze_video(full_path, emit_frame=True, emit_result=False)
            self.video_processed.emit(result)
        
class VideoProcessingThread(QThread):
    update_progress = pyqtSignal(int, int, str)
    update_result = pyqtSignal(str, list, dict)
    update_frame = pyqtSignal(np.ndarray)
    video_processed = pyqtSignal(dict)  # Новый сигнал для завершения обработки видео
    def __init__(self):
        super().__init__()
        self.video_path = None
        self.model = None
        self.image_size = (64, 64)  # Явная инициализация
        self.SEQUENCE_LENGTH = 16
        self.processing = True


        
    def run(self):
        for video_file in video_files:
            full_path = os.path.join(self.video_path, video_file)
            self.analyze_video(full_path)  # Вызов внутри потока
        video_files = [f for f in os.listdir(self.video_path) 
                      if f.lower().endswith(('.mp4', '.avi', '.mov'))]
        total_videos = len(video_files)
        
        for i, video_file in enumerate(video_files):
            if not self.processing:
                break
            self.update_progress.emit(i+1, total_videos, f"Обработка {video_file}")
            full_path = os.path.join(self.video_path, video_file)
            self.analyze_video(full_path)

    def analyze_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            self.video_processed.emit({
                "filename": os.path.basename(video_path),
                "status": "error",
                "total_frames": 0,
                "violence_frames": 0,
                "max_confidence": 0.0
             })
            return

            # Проверка на статичное видео
            ret, frame1 = cap.read()
            ret, frame2 = cap.read()
            if not ret or np.array_equal(frame1, frame2):
                cap.release()
                self.video_processed.emit({
                    "filename": os.path.basename(video_path),
                    "status": "static",
                    "total_frames": 0,
                    "violence_frames": 0,
                    "max_confidence": 0.0
                })
                return

        # Основной анализ кадров
        frame_sequence = []
        violence_timestamps = []
        max_confidence = 0  # Для хранения максимальной точности
    
        video_result = {
            "filename": os.path.basename(video_path),
            "total_frames": 0,
            "violence_frames": 0,
            "violence_percentage": 0.0,
            "timestamps": [],
            "max_confidence": 0  # Будем обновлять это значение
        }
    
        while cap.isOpened() and self.processing:
            ret, frame = cap.read()
            if not ret:
                break
            if emit_frame and self.update_frame:  # Добавлена проверка на update_frame
                self.update_frame.emit(frame.copy())
                self.update_frame.emit(frame.copy())
            #resized = cv2.resize(frame, (image_size[0], image_size[1]))
            # Используем self.image_size вместо image_size
            resized = cv2.resize(frame, (self.image_size[0], self.image_size[1]))  # <-- Исправлено здесь
            normalized = resized / 255.0
 
            frame_sequence.append(normalized)
        
            if len(frame_sequence) == self.SEQUENCE_LENGTH:
                input_data = np.expand_dims(frame_sequence, axis=0)
                prediction = self.model.predict(input_data, verbose=0)[0]
                is_violence = np.argmax(prediction) == 1
                confidence = float(prediction[1])
            
                # Обновляем максимальную точность
                if confidence > max_confidence:
                    max_confidence = confidence
                    video_result["max_confidence"] = max_confidence
            
                if is_violence:
                    current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                    timestamp = (current_frame - self.SEQUENCE_LENGTH) / fps
                    violence_timestamps.append(timestamp)
                    video_result["violence_frames"] += self.SEQUENCE_LENGTH
                
                    self.update_result.emit(
                        f"Насилие обнаружено на {timestamp:.1f} сек. (Точность: {confidence:.2%})",
                        violence_timestamps.copy(),
                        video_result
                    )
                
                frame_sequence = []
    
        cap.release()
        video_result["total_frames"] = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        video_result["violence_percentage"] = (
            (video_result["violence_frames"] / video_result["total_frames"]) * 100
            if video_result["total_frames"] > 0 else 0.0
        )
        video_result["max_confidence"] = max_confidence  # Сохраняем окончательное значение
        self.video_processed.emit(video_result)  # Отправляем результат

        # Основной класс приложения        
class ViolenceDetectorApp(QMainWindow):
    def __init__(self):
        super().__init__()  # Инициализация QMainWindow
        self.is_fullscreen = False
        MIN_CONFIDENCE_THRESHOLD = 50.0  # Общий порог для всего приложения
        self.SEQUENCE_LENGTH = 16  # Добавляем константу
            # Инициализация других атрибутов
        self.video_path = None
        self.current_file_data = {}
        self.results = []
        self.simplified_mode = False
        self.max_confidences = {}
        self.setup_ui()  # Настраиваем интерфейс после всей инициализации    
        try:
             #self.model = load_model('best_model_Unidirectional_LSTM.h5')
            #self.model = load_model('Bidirectional_LSTM_model.h5')
            self.model = load_model('bidirectional_lstm_model_20250505_050834.h5')
            
        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Не удалось загрузить модель:\n{str(e)}")
            sys.exit(1)
 
        self.thread = VideoProcessingThread()
        self.thread.video_processed.connect(self.handle_result)
    
    def _handle_frame_update(self, text, timestamps, result):
        """Обработчик промежуточных результатов в покадровом режиме"""
        self.console.append(text)
        self.current_file_data = result  # Сохраняем для последующего экспорта
        
    def resizeEvent(self, event):
        """Обработчик изменения размера окна"""
        if self.is_fullscreen:
            # Обновляем размер видео в полноэкранном режиме
            if hasattr(self, 'video_label') and self.video_label.pixmap():
                self.video_label.setPixmap(
                    self.video_label.pixmap().scaled(
                        self.video_label.size(),
                        Qt.KeepAspectRatioByExpanding
                    )
                )
        
        # Обновляем позицию кнопки выхода из полноэкранного режима
        if hasattr(self, 'exit_fullscreen_btn'):
            self.exit_fullscreen_btn.move(
                self.width() - self.exit_fullscreen_btn.width() - 20,
                self.height() - self.exit_fullscreen_btn.height() - 20
            )
        
        super().resizeEvent(event)  # Важно: вызываем родительский метод    
        
    def toggle_fullscreen(self):
        if self.is_fullscreen:
            self.showNormal()
            self.menuBar().show()
            self.left_panel.show()
            self.exit_fullscreen_btn.hide()
            self.is_fullscreen = False
        else:
            self.menuBar().hide()
            self.left_panel.hide()
            self.showFullScreen()
            self.exit_fullscreen_btn.show()
            self.exit_fullscreen_btn.raise_()
            self.is_fullscreen = True
        self.activateWindow()  # Возвращаем фокус
    
    def setup_ui(self):
        self.setWindowTitle("Распознавание насилия на видео")
        self.setGeometry(100, 100, 1200, 800)
        self.setStyleSheet(self.get_styles())
        self.setFont(QFont("Segoe UI", 10))

        # Создаем главный контейнер с горизонтальной компоновкой
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)
        main_layout.setContentsMargins(0, 0, 0, 0)
        main_layout.setSpacing(0)

        # Левая панель для логотипа (5x5 см)
        left_panel = QWidget()
        left_panel.setStyleSheet("background-color: #F0F0F0; order-right: 1px solid #d0d0d0;")
        left_layout = QVBoxLayout(left_panel)
        left_layout.setContentsMargins(10, 10, 10, 10)
        # Сохраняем ссылки на важные элементы
        self.left_panel = left_panel  # Добавляем эту строку
    
        # Добавляем горячую клавишу для полноэкранного режима (F11)
        self.fullscreen_action = QAction(self)
        self.fullscreen_action.setShortcut("F11")
        self.fullscreen_action.triggered.connect(self.toggle_fullscreen)
        self.addAction(self.fullscreen_action)
        # Рассчитываем размер логотипа (5 см в пикселях)
        screen = QApplication.primaryScreen()
        dpi = screen.logicalDotsPerInch()
        size_in_px = int(7 * (1 / 2.54) * dpi)  # Конвертируем см в пиксели

        # Загружаем логотип
        logo_label = QLabel()
        try:
            # Пробуем разные способы определения пути к файлу
            base_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
            logo_path = os.path.join(base_path, "CamShow.png")
        
            if os.path.exists(logo_path):
                logo_pixmap = QPixmap(logo_path)
                logo_pixmap = logo_pixmap.scaled(size_in_px, size_in_px, 
                                               Qt.KeepAspectRatio, Qt.SmoothTransformation)
                logo_label.setPixmap(logo_pixmap)
            else:
                # Если файл не найден, попробуем в текущей директории
                logo_path = "CamShow.png"
                if os.path.exists(logo_path):
                    logo_pixmap = QPixmap(logo_path)
                    logo_pixmap = logo_pixmap.scaled(size_in_px, size_in_px, 
                                                   Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    logo_label.setPixmap(logo_pixmap)
                else:
                    logo_label.setText("Логотип")
                    QMessageBox.warning(self, "Внимание", "Файл логотипа CamShow.png не найден!")
        except Exception as e:
            logo_label.setText("Ошибка загрузки логотипа")
            print(f"Ошибка при загрузке логотипа: {str(e)}")

        # Кнопка выхода из полноэкранного режима
        self.exit_fullscreen_btn = QPushButton("Exit Fullscreen (ESC)", self)
        self.exit_fullscreen_btn.setObjectName("exitFullscreenButton")
        self.exit_fullscreen_btn.setStyleSheet("""
            #exitFullscreenButton {
                background: rgba(70, 70, 70, 150);
                color: white;
                border: 1px solid #555;
                padding: 8px;
                border-radius: 4px;
                font: bold 12px;
            }
            #exitFullscreenButton:hover {
                background: rgba(90, 90, 90, 200);
            }
        """)
        self.exit_fullscreen_btn.hide()
        self.exit_fullscreen_btn.clicked.connect(self.toggle_fullscreen)
        self.exit_fullscreen_btn.resize(200, 40)
        logo_label.setAlignment(Qt.AlignCenter)
        logo_label.setStyleSheet("""
            QLabel {
                background: white;
                border: 1px solid #ccc;
                border-radius: 5px;
                padding: 5px;
            }
        """)
    
        # Устанавливаем фиксированную ширину левой панели
        left_panel.setFixedWidth(size_in_px + 30)  # +30 для отступов
    
        # Добавляем логотип в левую панель
        left_layout.addWidget(logo_label)
        left_layout.addStretch()  # Растягиваем пространство

        # Правая панель с основным содержимым
        right_panel = QWidget()
        right_layout = QVBoxLayout(right_panel)
        right_layout.setContentsMargins(10, 10, 10, 10)
    
        # Переносим существующие элементы в правую панель
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignCenter)
        self.video_label.setMinimumSize(640, 480)
        right_layout.addWidget(self.video_label)
    
        self.progress_bar = QProgressBar()
        right_layout.addWidget(self.progress_bar)
    
        self.console = QTextEdit()
        self.console.setReadOnly(True)
        right_layout.addWidget(self.console)
    
        # Добавляем обе панели в главный layout
        main_layout.addWidget(left_panel)
        main_layout.addWidget(right_panel)
    
        # Настраиваем пропорции растяжения
        main_layout.setStretch(0, 1)  # Левая панель
        main_layout.setStretch(1, 5)  # Правая панель
    
        self.create_menu()
                     
    def get_styles(self):
        return """
            QMainWindow {
                background-color: #F5F5F5;
            }
            QLabel {
                font: 14px 'Segoe UI';
                color: #333333;
            }
            QProgressBar {
                border: 1px solid #CCCCCC;
                border-radius: 5px;
                height: 25px;
                text-align: center;
                background: #FFFFFF;
            }
            QProgressBar::chunk {
                background-color: #4CAF50;
                border-radius: 5px;
            }
            QTextEdit {
                background: #FFFFFF;
                border: 1px solid #CCCCCC;
                border-radius: 5px;
                font: 12px 'Consolas';
                color: #333333;
            }
            QMenuBar {
                background: #FFFFFF;
                border-bottom: 1px solid #CCCCCC;
            }
            QMenuBar::item {
                padding: 5px 10px;
                color: #444444;
            }
            QMenuBar::item:selected {
                background: #E0E0E0;
            }
            QMenu {
                background: #FFFFFF;
                border: 1px solid #CCCCCC;
            }
            QMenu::item:selected {
                background: #4CAF50;
                color: #FFFFFF;
            }
            QMainWindow:fullscreen {
                background-color: black;
            }
    
            QLabel:fullscreen {
                border: none;
            }
            """

    def create_menu(self):
        menubar = self.menuBar()
        
        # Главное меню "Распознавание насилия"
        recognition_menu = menubar.addMenu("Распознавание насилия")
        
        # Пункты меню для разных режимов обработки
        frame_by_frame_action = QAction("Покадровое распознавание", self)
        frame_by_frame_action.triggered.connect(self.select_folder_frame_by_frame)
        recognition_menu.addAction(frame_by_frame_action)
        
        file_by_file_action = QAction("Пофайловое распознавание", self)
        file_by_file_action.triggered.connect(self.select_folder_file_by_file)
        recognition_menu.addAction(file_by_file_action)
        
        # Меню "Экспорт" (без "Выбрать папку")
        file_menu = menubar.addMenu("Экспорт")
        export_action = QAction("Экспорт результатов...", self)
        export_action.triggered.connect(self.export_results)
        file_menu.addAction(export_action)
        
        exit_action = QAction("Выход", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)
        
        # Меню "Статистика"
        stats_menu = menubar.addMenu("Статистика")
        load_stats_action = QAction("Загрузить из экспорта", self)
        load_stats_action.triggered.connect(self.load_statistics)
        stats_menu.addAction(load_stats_action)
        
        # Меню "Обработка"
        process_menu = menubar.addMenu("Обработка")
        stop_action = QAction("Остановить", self)
        stop_action.triggered.connect(self.stop_processing)
        process_menu.addAction(stop_action)
    
    @pyqtSlot(np.ndarray)
    def update_video_frame(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame.shape
        bytes_per_line = ch * w
        q_img = QImage(frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        if self.is_fullscreen:
            # В полноэкранном режиме растягиваем на весь экран
            pixmap = QPixmap.fromImage(q_img).scaled(
                self.width(),
                self.height(),
                Qt.KeepAspectRatioByExpanding
            )
        else:
            # В обычном режиме сохраняем пропорции
            pixmap = QPixmap.fromImage(q_img).scaled(
                self.video_label.width(),
                self.video_label.height(),
                Qt.KeepAspectRatio
            )
        #pixmap = QPixmap.fromImage(q_img).scaled(
        #    self.video_label.width(),
        #    self.video_label.height(),
        #    Qt.KeepAspectRatio
        #)
        self.video_label.setPixmap(pixmap)

    @pyqtSlot(int, int, str)
    def update_progress(self, current, total, message):
        self.progress_bar.setValue(int((current / total) * 100))
        self.console.append(f"[{datetime.now().strftime('%H:%M:%S')}] {message}")

    @pyqtSlot(str, list, dict)
    def handle_frame_result(self, text, timestamps, result):
        """Обработка результатов покадрового режима"""
        # Инициализация данных при начале обработки нового файла
        if result["filename"] != self.current_file_data.get("filename"):
            self.current_file_data = {
                "filename": result["filename"],
                "entries": [],
                "total_frames": result["total_frames"],
                "violence_frames": 0,
                "max_confidence": 0.0
            }
    
        # Обновление данных
        self.current_file_data["violence_frames"] = result["violence_frames"]
        self.current_file_data["max_confidence"] = max(
            self.current_file_data["max_confidence"], 
            result.get("max_confidence", 0)
        )
        # Обновление максимальной точности из result
        self.current_file_data["max_confidence"] = max(
            self.current_file_data["max_confidence"],
            result.get("max_confidence", 0)
        )    
        # Парсинг данных из текстового сообщения
        time_part = float(text.split("сек.")[0].split("на ")[-1].strip())  # Добавлен float()
        confidence = float(text.split("Точность: ")[-1].replace("%)", ""))
        self.current_file_data["entries"].append(
            (time_part, confidence)
        )
        """Обработка результатов покадрового режима"""
        self.console.append(text)
        self.console.append(f"Найдено эпизодов: {len(timestamps)}")
        self.results.append(result)
        #self.console.append(text)
        #status = "обнаружено" if result["violence_frames"] > 0 else "не обнаружено"
        #final_msg = f"\nАнализ завершен. Насилие {status}."
        #final_msg = f"\nАнализ завершен. Насилие {status}."
        #self.console.append(final_msg)
    
    def select_folder_frame_by_frame(self):
        """Режим покадрового анализа"""
        self.simplified_mode = False
        self.console.clear()

        # Настраиваем диалог выбора с отображением файлов
        dialog = QFileDialog()
        dialog.setFileMode(QFileDialog.Directory)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)  # Важно!
        dialog.setNameFilter("Видео файлы (*.mp4 *.avi *.mov)")
        dialog.setWindowTitle("Выберите папку с видеофайлами")
    
        # Для QFileDialog в не-нативном режиме можно настроить отображение файлов
        if hasattr(dialog, 'setOption'):
            dialog.setOption(QFileDialog.DontUseNativeDialog, True)
    
        # Получаем список файлов в выбранной папке
        if dialog.exec_():
            folder = dialog.selectedFiles()[0]
            if folder:
                # Проверяем наличие видеофайлов
                video_files = [f for f in os.listdir(folder) 
                             if f.lower().endswith(('.mp4', '.avi', '.mov'))]
            
                if not video_files:
                    QMessageBox.warning(self, "Ошибка", 
                                      f"В папке {folder} нет видеофайлов (MP4, AVI, MOV)!")
                    return
                
                self.video_path = folder
                self.console.append(f"Выбрана папка: {folder}")
                self.console.append(f"Найдено видеофайлов: {len(video_files)}")
                self.process_videos()

    def select_folder_file_by_file(self):
        """Режим пофайлового анализа"""
        self.simplified_mode = True
        self.max_confidences = {}
        self.console.clear()

        dialog = QFileDialog()
        dialog.setFileMode(QFileDialog.Directory)
        dialog.setOption(QFileDialog.DontUseNativeDialog, True)
        dialog.setNameFilter("Видео файлы (*.mp4 *.avi *.mov)")
        dialog.setWindowTitle("Выберите папку с видеофайлами")
    
        if dialog.exec_():
            folder = dialog.selectedFiles()[0]
            if folder:
                video_files = [f for f in os.listdir(folder) 
                             if f.lower().endswith(('.mp4', '.avi', '.mov'))]
            
                if not video_files:
                    QMessageBox.warning(self, "Ошибка",
                                      f"В папке {folder} нет поддерживаемых видеофайлов!")
                    return
                
                self.video_path = folder
                self.console.append(f"Выбрана папка: {folder}")
                self.console.append(f"Найдено видеофайлов: {len(video_files)}")
                self.process_videos()
                
    def process_videos_simplified(self):
        """Упрощенная обработка видео с выводом только максимальной точности"""
        if not self.video_path:
            return
        
        self.thread = VideoProcessingThread()
        self.thread.video_path = self.video_path
        self.thread.model = self.model
        self.thread.image_size = (64, 64)
        self.thread.SEQUENCE_LENGTH = 16

        # Подключаем новые сигналы
        self.thread.video_processed.connect(self.handle_video_processed)  # <-- Добавлено
        self.thread.finished.connect(self.on_simplified_processing_finished)  # <-- Добавлено

        # Остальные подключения сигналов
        self.thread.update_progress.connect(self.update_progress)
        self.thread.update_result.connect(self.update_result)
        self.thread.update_frame.connect(self.update_video_frame)

        self.thread.start()
       
        
    def on_processing_finished(self):
        """Действия после завершения обработки"""
        if self.simplified_mode:
            # Вывод итоговых результатов для упрощенного режима
            self.console.append("\nИтоговые результаты:\n")
        
            for filename, max_confidence in sorted(self.max_confidences.items()):
                if max_confidence == 0:
                    self.console.append(f"{filename}: Насилие не обнаружено")
                else:
                    description = self.get_confidence_description(max_confidence)
                    self.console.append(
                        f"{filename}: Максимальная точность {max_confidence:.2f}% ({description})"
                    )
        
            # Сохраняем результаты для экспорта
            self.results = [{
                "filename": filename,
                "max_confidence": max_confidence,
                "description": "Насилие не обнаружено" if max_confidence == 0 
                              else self.get_confidence_description(max_confidence)
            } for filename, max_confidence in self.max_confidences.items()]
    
        self.console.append("\nОбработка всех видео завершена\n")

    def analyze_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_sequence = []
        violence_timestamps = []
        max_confidence = 0  # Для хранения максимальной точности
        
        video_result = {
            "filename": os.path.basename(video_path),
            "total_frames": 0,
            "violence_frames": 0,
            "violence_percentage": 0.0,
            "timestamps": [],
            "max_confidence": 0
        }
        
        while cap.isOpened() and self.processing:
            ret, frame = cap.read()
            if not ret:
                break
                
            if not self.simplified_mode:
                self.update_frame.emit(frame.copy())
                
            resized = cv2.resize(frame, (self.image_size[0], self.image_size[1]))
            normalized = resized / 255.0
            frame_sequence.append(normalized)
            
            if len(frame_sequence) == self.SEQUENCE_LENGTH:
                input_data = np.expand_dims(frame_sequence, axis=0)
                prediction = self.model.predict(input_data, verbose=0)[0]
                is_violence = np.argmax(prediction) == 1
                confidence = float(prediction[1]) * 100  # Преобразуем в проценты
                
                # Обновляем максимальную точность
                if confidence > max_confidence:
                    max_confidence = confidence
                    video_result["max_confidence"] = max_confidence
                
                if is_violence:
                    current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                    timestamp = (current_frame - self.SEQUENCE_LENGTH) / fps
                    violence_timestamps.append(timestamp)
                    video_result["violence_frames"] += self.SEQUENCE_LENGTH
                    
                    if not self.simplified_mode:
                        self.update_result.emit(
                            f"Насилие обнаружено на {timestamp:.1f} сек. (Точность: {confidence:.2f}%)",
                            violence_timestamps.copy(),
                            video_result
                        )
                
                frame_sequence = []
        
        cap.release()
        video_result["total_frames"] = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        video_result["violence_percentage"] = (
            (video_result["violence_frames"] / video_result["total_frames"]) * 100
            if video_result["total_frames"] > 0 else 0.0
        )
        video_result["max_confidence"] = max_confidence
        
        # Для упрощенного режима сохраняем максимальную точность
        if self.simplified_mode:
            filename = video_result["filename"]
            if filename not in self.max_confidences or max_confidence > self.max_confidences[filename]:
                self.max_confidences[filename] = max_confidence
        self.video_file_processed.emit()  # <-- Добавьте эту строку в конец метода
        
    def on_simplified_processing_finished(self):
        """Вывод результатов после завершения упрощенной обработки"""
        self.console.append("\nИтоговые результаты:\n")
    
        # Сортируем файлы по имени
        sorted_results = sorted(self.max_confidences.items(), key=lambda x: x[0])
    
        for filename, max_confidence in sorted_results:
            description = self.get_confidence_description(max_confidence)
            if max_confidence == 0:
                self.console.append(f"{filename}: {description}")
            else:
                self.console.append(
                    f"{filename}: Максимальная точность {max_confidence:.2f}% ({description})"
                )
    
        self.console.append("\nОбработка всех видео завершена\n")
    
        # Обновляем результаты для экспорта
        self.results = [{
            "filename": filename,
            "max_confidence": max_confidence,
            "description": description
        } for filename, max_confidence in self.max_confidences.items()]

        # УДАЛИТЬ ниже в этой функции ? 
        # Подключаем только нужные сигналы
        self.thread.update_progress.connect(self.update_progress)
        self.thread.finished.connect(self.on_simplified_processing_finished)
    
        # Для упрощенного режима нам нужно хранить максимальные значения
        self.max_confidences = {}  # {filename: max_confidence}
        self.current_file = None
        self.thread.start()
    
    def get_confidence_description(self, confidence):
        """Возвращает текстовое описание уровня уверенности"""
        if confidence == 0:
            return "Насилие не обнаружено"
        elif 1 <= confidence <= 10:
            return "Маловероятно насилие"
        elif 11 <= confidence <= 30:
            return "Низкая вероятность насилия"
        elif 31 <= confidence <= 60:
            return "Средняя вероятность насилия"
        elif 61 <= confidence <= 80:
            return "Высокая вероятность насилия"
        else:
            return "Точно насилие"
        
    def _print_current_file_report(self):
        """Форматирование и вывод отчета по текущему файлу"""
        if not self.current_file_data:
            return
    
        # Заголовок
        self.console.append(f"\n{'-'*40}")
        self.console.append(
            f"Экспорт: {self.current_file_data['filename']}\n"
            f"{'№ Времени':<15} {'Точность':<10}"
        )
    
        # Данные таблицы
        for idx, (time, conf) in enumerate(self.current_file_data["entries"], 1):
            self.console.append(f"{idx:<15} {time:<8} сек. | {conf:.2f}%")
    
        # Итоговая информация
        self.console.append("\nИтоги:")
        self.console.append(
            f"Всего кадров: {self.current_file_data['total_frames']}\n"
            f"Кадров с насилием: {self.current_file_data['violence_frames']}\n"
            f"Максимальная точность: {self.current_file_data['max_confidence']:.2f}%"
        )
        self.console.append(f"{'-'*40}\n")
    
        # Сохранение в общие результаты
        self.results.append(self.current_file_data)
        self.current_file_data = {}
        
    def select_folder(self):
        self.console.clear()  # Очищаем консоль перед новым анализом
        folder = QFileDialog.getExistingDirectory(self, "Выберите папку с видео")
        if folder:
            self.video_path = folder
            self.process_videos()

    def select_preprocessing_folder(self):
        input_dir = QFileDialog.getExistingDirectory(self, "Выберите папку с исходными видео")
        if not input_dir:
            return
            
        output_dir = QFileDialog.getExistingDirectory(self, "Выберите папку для сохранения")
        if not output_dir:
            return
            
        self.console.clear()
        self.console.append("Начата предобработка видео...")
        
        self.preprocess_thread = VideoPreprocessingThread(
            input_dir=input_dir,
            output_dir=output_dir,
            image_size=(64, 64),
            sequence_length=16
        )
        
        self.preprocess_thread.progress_signal.connect(self.update_preprocess_progress)
        self.preprocess_thread.finished_signal.connect(self.preprocessing_finished)
        self.preprocess_thread.start()
       
    def update_preprocess_progress(self, progress, message):
        self.progress_bar.setValue(progress)
        self.console.append(message)

    def preprocessing_finished(self, message):
        self.console.append(message)
        QMessageBox.information(self, "Успех", "Предобработка видео завершена!")

    # В классе ViolenceDetectorApp обновляем метод handle_video_processed

    @pyqtSlot(dict)
    def handle_result(self, result):
        """Общий обработчик результатов (для обоих режимов)"""
        if self.simplified_mode:
            self.handle_file_result(result)
        else:
            self._handle_frame_result(result)
    
        # Сохраняем результат для экспорта
        self.results.append(result)
        
    def handle_file_result(self, result):
        """Обработчик для пофайлового режима"""
        filename = result["filename"]
        if result.get("status") == "static":
            self.max_confidences[filename] = 0.0
            self.console.append(f"{filename}: статичное видео (пропущено)")
            return
    
        max_conf = result["max_confidence"]
        violence_frames = result["violence_frames"]
    
        if violence_frames > 0 and max_conf >= self.MIN_CONFIDENCE_THRESHOLD:
            self.max_confidences[filename] = max_conf
            self.console.append(f"{filename}: обнаружено насилие (точность: {max_conf:.2f}%)")
        else:
            self.max_confidences[filename] = 0.0
            self.console.append(f"{filename}: насилие не обнаружено")
    def _handle_frame_result(self, result):
        """Обработчик для покадрового режима"""
        # Существующая логика обработки покадровых результатов
        if result.get("status") == "static":
            self.console.append(f"{result['filename']}: пропущено (статичное видео)")
        else:
            filename = result["filename"]
            max_conf = result["max_confidence"]
            violence_frames = result["violence_frames"]
            
            if violence_frames > 0:
                self.console.append(
                    f"{filename}: обнаружено насилие (кадров: {violence_frames}, "
                    f"макс. точность: {max_conf:.2f}%)"
                )
            else:
                self.console.append(f"{filename}: насилие не обнаружено")
            
            # Для покадрового режима сохраняем полные данные
            self.results.append(result)
    def process_videos(self):
        if not self.video_path:
            return
    
        # Проверяем наличие видеофайлов
        video_files = [f for f in os.listdir(self.video_path)
                      if f.lower().endswith(('.mp4', '.avi', '.mov'))]
    
        if not video_files:
            QMessageBox.warning(self, "Ошибка",
                              "В выбранной папке нет видеофайлов (MP4, AVI, MOV)!")
            return
    
        # Продолжаем обработку
        if self.simplified_mode:
            self.thread = FileAnalysisThread(
                self.video_path,
                self.model,
                (64, 64),
                16
            )
            self.thread.video_processed.connect(self.handle_result)
        else:
            self.thread = FrameAnalysisThread(
                self.video_path,
                self.model,
                (64, 64),
                16
            )
            self.thread.video_file_processed.connect(self._print_current_file_report)
            self.thread.update_result.connect(self.handle_frame_result)
    
        # Общие подключения сигналов
        self.thread.update_progress.connect(self.update_progress)
        self.thread.update_frame.connect(self.update_video_frame)
        self.thread.finished.connect(lambda: self.console.append("\nОбработка всех видео завершена\n"))
    
        self.thread.start()
 
    def stop_processing(self):
        if hasattr(self, 'thread') and self.thread.isRunning():
            self.thread.processing = False
            self.thread.quit()
            self.console.append("\n[Остановлено пользователем]\n")
             
    def load_statistics(self, _=None):
        file_path, _ = QFileDialog.getOpenFileName(
            self,
            "Выберите файл экспорта",
            "",
            "Excel/JSON Files (*.xlsx *.json)"
        )

        if not file_path:
            return

        try:
            dialog = QDialog(self)
            dialog.setWindowTitle("Просмотр экспорта")
            dialog.setMinimumSize(800, 600)
            layout = QVBoxLayout()

            if file_path.endswith('.json'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                text_edit = QTextEdit()
                text_edit.setReadOnly(True)
                text_edit.setPlainText(json.dumps(data, indent=4, ensure_ascii=False))
                layout.addWidget(text_edit)
            else:
                # Используем openpyxl напрямую для чтения Excel
                from openpyxl import load_workbook
                wb = load_workbook(filename=file_path, read_only=True)
                ws = wb.active
            
                table = QTableWidget()
                table.setRowCount(ws.max_row - 1)  # Исключаем заголовок
                table.setColumnCount(ws.max_column)

                # Заполнение заголовков
                headers = []
                for col in range(1, ws.max_column + 1):
                    header_value = ws.cell(row=1, column=col).value
                    headers.append(str(header_value) if header_value is not None else "")
                table.setHorizontalHeaderLabels(headers)

                # Заполнение данных
                for row in range(2, ws.max_row + 1):  # Начинаем с второй строки
                    for col in range(1, ws.max_column + 1):
                        cell = ws.cell(row=row, column=col)
                        # Обработка пустых значений и NaN
                        cell_value = cell.value
                        if cell_value is None:
                            display_value = ""
                        elif isinstance(cell_value, float) and np.isnan(cell_value):
                            display_value = ""
                        else:
                            display_value = str(cell_value)
                    
                        item = QTableWidgetItem(display_value)
                        item.setFlags(item.flags() & ~Qt.ItemIsEditable)
                        table.setItem(row-2, col-1, item)  # row-2 из-за смещения заголовка

                # Настройка стилей
                table.setStyleSheet("""
                    QTableWidget {
                        gridline-color: #e0e0e0;
                        font: 12px 'Segoe UI';
                    }
                    QHeaderView::section {
                        background-color: #f0f0f0;
                        padding: 5px;
                        border: 1px solid #d0d0d0;
                    }
                """)
            
                # Автоподбор ширины столбцов
                table.horizontalHeader().setSectionResizeMode(QHeaderView.ResizeToContents)
                table.horizontalHeader().setStretchLastSection(True)
                table.verticalHeader().setVisible(False)
                table.setAlternatingRowColors(True)
            
                # Дополнительная настройка для первой колонки
                table.resizeColumnsToContents()
                if table.columnCount() > 0:
                    # Увеличиваем ширину первой колонки на 20% от содержимого
                    width = table.columnWidth(0)
                    table.setColumnWidth(0, int(width * 1.2))

                layout.addWidget(table)

            dialog.setLayout(layout)
            dialog.exec_()

        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка загрузки:\n{str(e)}")
        
    def export_results(self):
        if not self.results and not self.max_confidences:
            QMessageBox.warning(self, "Ошибка", "Нет данных для экспорта!")
            return

        try:
            # Импортируем необходимые классы из openpyxl
            from openpyxl import Workbook
            from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
            from openpyxl.utils import get_column_letter

            wb = Workbook()
            ws = wb.active
            ws.title = "Результаты"

            # Определяем стили
            thin_border = Border(left=Side(style='thin'),
                             right=Side(style='thin'),
                             top=Side(style='thin'),
                             bottom=Side(style='thin'))
            header_font = Font(bold=True, color="FFFFFF")
            header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            summary_font = Font(bold=True, size=12)
            file_header_font = Font(bold=True, italic=True)

            # Собираем статистику
            all_files = self._collect_all_files_data()
            total_files = len(all_files)
            violent_files = sum(1 for data in all_files.values() 
                              if data['max_conf'] > 0)  # Считаем файлы с любой уверенностью > 0

            # Добавляем общую статистику
            ws.append([f"Проверено {total_files} файлов, из них {violent_files} с признаками насилия"])
            ws['A1'].font = summary_font
            ws.merge_cells('A1:C1')
            ws.append([])  # Пустая строка

            row_idx = 3  # Начинаем с 3 строки
            for filename, data in all_files.items():
                # Заголовок файла
                ws.merge_cells(start_row=row_idx, start_column=1, end_row=row_idx, end_column=3)
                ws.cell(row=row_idx, column=1, value=f"Файл: {filename}").font = file_header_font
                row_idx += 1

                # Для покадрового режима
                if not self.simplified_mode and data['violence']:
                    headers = ["№", "Время (сек)", "Точность (%)"]
                    ws.append(headers)

                    # Применяем стили к заголовкам
                    for col in range(1, 4):
                        cell = ws.cell(row=row_idx, column=col)
                        cell.font = header_font
                        cell.fill = header_fill
                        cell.border = thin_border
                    row_idx += 1

                    # Добавляем данные
                    for idx, (time, conf) in enumerate(data['entries'], 1):
                        ws.append([idx, float(time), float(conf)])
                        row_idx += 1

                # Итоги
                ws.merge_cells(start_row=row_idx, start_column=1, end_row=row_idx, end_column=3)
                ws.cell(row=row_idx, column=1, value="Итоги:").font = Font(bold=True)
                row_idx += 1

                stats = [
                    f"Всего кадров: {data['total_frames']}",
                    f"Кадров с насилием: {data['violence_frames']}",
                    f"Максимальная точность: {data['max_conf']:.2f}%"
                ]
                for stat in stats:
                    ws.cell(row=row_idx, column=1, value=stat)
                    row_idx += 1

                row_idx += 1  # Разделитель

            # Автоподбор ширины
            for col in ws.columns:
                col_letter = get_column_letter(col[0].column)
                max_len = max(len(str(cell.value)) for cell in col if cell.value) if any(cell.value for cell in col) else 0
                ws.column_dimensions[col_letter].width = max_len + 2

            # Генерация имени файла
            timestamp = datetime.now().strftime("%d-%m-%Y_%Hh.%Mm")
            mode_prefix = "Frame_" if not self.simplified_mode else "File_"
            filename = f"{mode_prefix}{timestamp}.xlsx"

            # Используем папку с видео как папку для сохранения по умолчанию
            save_dir = self.video_path if self.video_path else os.path.expanduser("~")

            # Диалог сохранения файла
            file_dialog = QFileDialog()
            file_dialog.setDefaultSuffix("xlsx")
            file_dialog.setNameFilter("Excel Files (*.xlsx)")
            file_dialog.setAcceptMode(QFileDialog.AcceptSave)
            file_dialog.setDirectory(save_dir)
            file_dialog.selectFile(filename)

            if file_dialog.exec_():
                selected_files = file_dialog.selectedFiles()
                if selected_files:
                    file_path = selected_files[0]
                    wb.save(file_path)
                    QMessageBox.information(self, "Успех", f"Файл экспортирован:\n{file_path}")

        except Exception as e:
            QMessageBox.critical(self, "Ошибка", f"Ошибка экспорта:\n{str(e)}")

    def is_valid_video(video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return False
    
        # Проверяем, что видео содержит движущиеся кадры
        ret, frame1 = cap.read()
        ret, frame2 = cap.read()
        cap.release()
    
        if not ret:
            return False
    
        # Сравниваем первые два кадра
        diff = cv2.absdiff(frame1, frame2)
        non_zero = np.count_nonzero(diff)
    
        return non_zero > (frame1.size * 0.01)  # Хотя бы 1% пикселей отличаются        
            
    def _collect_all_files_data(self):
        """Собирает данные по всем файлам в единую структуру."""
        all_files = {}

        # 1. Сначала собираем все файлы из результатов покадрового анализа
        for res in self.results:
            filename = res.get('filename')
            all_files[filename] = {
                'violence': res.get('violence_frames', 0) > 0,
                'entries': [(float(t[0]), float(t[1])) for t in res.get('entries', [])],
                'total_frames': res.get('total_frames', 0),
                'violence_frames': res.get('violence_frames', 0),
                'max_conf': res.get('max_confidence', 0)
            }

        # 2. Добавляем файлы из упрощенного анализа, включая те, где насилия нет
        for filename, max_conf in self.max_confidences.items():
            if filename not in all_files:
                all_files[filename] = {
                    'violence': max_conf > 0,  # Любое значение > 0 считаем потенциальным насилием
                    'entries': [],
                    'total_frames': 0,  # В пофайловом режиме не знаем общее количество кадров
                    'violence_frames': self.SEQUENCE_LENGTH if max_conf > 0 else 0,
                    'max_conf': max_conf
                }

        # 3. Добавляем файлы, которые были обработаны, но не попали ни в один из результатов
        if self.video_path:
            all_video_files = [f for f in os.listdir(self.video_path) 
                              if f.lower().endswith(('.mp4', '.avi', '.mov'))]
            for filename in all_video_files:
                if filename not in all_files:
                    all_files[filename] = {
                        'violence': False,
                        'entries': [],
                        'total_frames': 0,
                        'violence_frames': 0,
                        'max_conf': 0.0
                    }

        return all_files

    @pyqtSlot(dict)
    def handle_file_result(self, result):
        """Обработка результатов общего режима"""
        filename = result["filename"]
        max_conf = result["max_confidence"]
        self.console.append(f"{filename}: Макс. точность - {max_conf:.2f}%")
        # Сохраняем результат для экспорта
        self.max_confidences[filename] = max_conf

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ViolenceDetectorApp()
    window.show()
    sys.exit(app.exec_())
